# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 17 2023 11:43AM,255220,20,9042407,"Exact-Rx, Inc.",Released
1,Jan 17 2023 11:40AM,255217,10,MSP217614,Methapharm-G,Released
2,Jan 17 2023 11:40AM,255217,10,MSP217617,Methapharm-G,Released
3,Jan 17 2023 11:40AM,255217,10,MSP217636,"Methapharm, Inc.",Released
4,Jan 17 2023 11:40AM,255217,10,MSP217616,"Methapharm, Inc.",Released
5,Jan 17 2023 11:40AM,255217,10,MSP217627,"Methapharm, Inc.",Released
6,Jan 17 2023 11:40AM,255217,10,MSP217634,"Methapharm, Inc.",Released
7,Jan 17 2023 11:39AM,255216,10,MSP217635,"Methapharm, Inc.",Released
8,Jan 17 2023 11:39AM,255216,10,MSP217628,"Methapharm, Inc.",Released
9,Jan 17 2023 11:39AM,255216,10,MSP217629,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,255212,Released,1
34,255215,Released,4
35,255216,Released,8
36,255217,Released,6
37,255220,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
255212,NaN,NaN,1.0
255215,NaN,NaN,4.0
255216,NaN,NaN,8.0
255217,NaN,NaN,6.0
255220,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
255097,0.0,42.0,2.0
255098,0.0,0.0,22.0
255157,0.0,22.0,78.0
255162,0.0,0.0,1.0
255172,0.0,0.0,21.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
255097,0,42,2
255098,0,0,22
255157,0,22,78
255162,0,0,1
255172,0,0,21


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,255097,0,42,2
1,255098,0,0,22
2,255157,0,22,78
3,255162,0,0,1
4,255172,0,0,21


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,255097,,42,2
1,255098,,,22
2,255157,,22,78
3,255162,,,1
4,255172,,,21


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 17 2023 11:43AM,255220,20,"Exact-Rx, Inc."
1,Jan 17 2023 11:40AM,255217,10,Methapharm-G
3,Jan 17 2023 11:40AM,255217,10,"Methapharm, Inc."
7,Jan 17 2023 11:39AM,255216,10,"Methapharm, Inc."
15,Jan 17 2023 11:37AM,255215,10,"Snap Medical Industries, LLC"
19,Jan 17 2023 11:26AM,255212,10,Yusen – 3D Matrix
20,Jan 17 2023 11:18AM,255211,19,ACG North America LLC
21,Jan 17 2023 11:16AM,255210,19,ACG North America LLC
22,Jan 17 2023 11:10AM,255209,19,"GUSA Granules USA, Inc."
23,Jan 17 2023 11:06AM,255208,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 17 2023 11:43AM,255220,20,"Exact-Rx, Inc.",,,1
1,Jan 17 2023 11:40AM,255217,10,Methapharm-G,,,6
2,Jan 17 2023 11:40AM,255217,10,"Methapharm, Inc.",,,6
3,Jan 17 2023 11:39AM,255216,10,"Methapharm, Inc.",,,8
4,Jan 17 2023 11:37AM,255215,10,"Snap Medical Industries, LLC",,,4
5,Jan 17 2023 11:26AM,255212,10,Yusen – 3D Matrix,,,1
6,Jan 17 2023 11:18AM,255211,19,ACG North America LLC,,,1
7,Jan 17 2023 11:16AM,255210,19,ACG North America LLC,,,1
8,Jan 17 2023 11:10AM,255209,19,"GUSA Granules USA, Inc.",,,1
9,Jan 17 2023 11:06AM,255208,10,"Citieffe, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 17 2023 11:43AM,255220,20,"Exact-Rx, Inc.",1,,
1,Jan 17 2023 11:40AM,255217,10,Methapharm-G,6,,
2,Jan 17 2023 11:40AM,255217,10,"Methapharm, Inc.",6,,
3,Jan 17 2023 11:39AM,255216,10,"Methapharm, Inc.",8,,
4,Jan 17 2023 11:37AM,255215,10,"Snap Medical Industries, LLC",4,,
5,Jan 17 2023 11:26AM,255212,10,Yusen – 3D Matrix,1,,
6,Jan 17 2023 11:18AM,255211,19,ACG North America LLC,1,,
7,Jan 17 2023 11:16AM,255210,19,ACG North America LLC,1,,
8,Jan 17 2023 11:10AM,255209,19,"GUSA Granules USA, Inc.",1,,
9,Jan 17 2023 11:06AM,255208,10,"Citieffe, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 17 2023 11:43AM,255220,20,"Exact-Rx, Inc.",1,,
1,Jan 17 2023 11:40AM,255217,10,Methapharm-G,6,,
2,Jan 17 2023 11:40AM,255217,10,"Methapharm, Inc.",6,,
3,Jan 17 2023 11:39AM,255216,10,"Methapharm, Inc.",8,,
4,Jan 17 2023 11:37AM,255215,10,"Snap Medical Industries, LLC",4,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 17 2023 11:43AM,255220,20,"Exact-Rx, Inc.",1,NaN,NaN
1,Jan 17 2023 11:40AM,255217,10,Methapharm-G,6,NaN,NaN
2,Jan 17 2023 11:40AM,255217,10,"Methapharm, Inc.",6,NaN,NaN
3,Jan 17 2023 11:39AM,255216,10,"Methapharm, Inc.",8,NaN,NaN
4,Jan 17 2023 11:37AM,255215,10,"Snap Medical Industries, LLC",4,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 17 2023 11:43AM,255220,20,"Exact-Rx, Inc.",1,0.0,0.0
1,Jan 17 2023 11:40AM,255217,10,Methapharm-G,6,0.0,0.0
2,Jan 17 2023 11:40AM,255217,10,"Methapharm, Inc.",6,0.0,0.0
3,Jan 17 2023 11:39AM,255216,10,"Methapharm, Inc.",8,0.0,0.0
4,Jan 17 2023 11:37AM,255215,10,"Snap Medical Industries, LLC",4,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4338311,143,58.0,1.0
12,765591,3,0.0,0.0
15,765417,101,22.0,0.0
19,2041622,29,0.0,0.0
20,510418,2,0.0,0.0
22,255187,1,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4338311,143,58.0,1.0
1,12,765591,3,0.0,0.0
2,15,765417,101,22.0,0.0
3,19,2041622,29,0.0,0.0
4,20,510418,2,0.0,0.0
5,22,255187,1,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,143,58.0,1.0
1,12,3,0.0,0.0
2,15,101,22.0,0.0
3,19,29,0.0,0.0
4,20,2,0.0,0.0
5,22,1,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,143.0
1,12,Released,3.0
2,15,Released,101.0
3,19,Released,29.0
4,20,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,22
Status,,,,,,
Completed,1.0,0.0,0.0,0.0,0.0,0.0
Executing,58.0,0.0,22.0,0.0,0.0,0.0
Released,143.0,3.0,101.0,29.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,22
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0
1,Executing,58.0,0.0,22.0,0.0,0.0,0.0
2,Released,143.0,3.0,101.0,29.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,22
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0
1,Executing,58.0,0.0,22.0,0.0,0.0,0.0
2,Released,143.0,3.0,101.0,29.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()